As of now, the dataset includes ~700 fastq files, even though there are only about 40 samples. This makes GEO submission and downsampling a pain. Therefore, I will generate 1 set of fastq per experiment. Everything will be documented here for transparency regarding these fastq.

We start off with the following files:

In [1]:
ls original_fastq

BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L001_R1_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L001_R2_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L001_R3_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L002_R1_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L002_R2_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L002_R3_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S2_L001_R1_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S2_L001_R2_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S2_L001_R3_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S2_L002_R1_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S2_L002_R2_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S2_L002_R3_001.fastq.gz
BAP__1d7912__10x_scATAC_PBMC

In [2]:
ls original_fastq | wc -l

701


Some of these runs have been co-sequenced. If a 10x scATAC run is co-sequenced with a 10x multiome run, then the resulting fastq will have a barcode read of 24 instead of 16. This will give problems when barcode-demultiplexing. Therefore, I cut only the relevant barcode part out for these particular runs.

In [10]:
# using start and end coordinates
module load pigz
split_fastq (){
    local input_fastq_filename="${1}";
    local output_fastq_split1_filename="${2}";
    local split1_start=1;
    local split1_end=16;
    zcat "${input_fastq_filename}" \
      | awk \
            -v "split1_start=${split1_start}" \
            -v "split1_end=${split1_end}" \
            -v "output_fastq_split1_filename=${output_fastq_split1_filename}" \
            '
            BEGIN {
                split1_length = split1_end - split1_start + 1;
                write_compressed_fastq_cmd = "pigz -p 2 -6 -c > " output_fastq_split1_filename;
            }
            {
                if (NR % 2 == 1) {
                    # Read name or "+" line.
                    print $0  | write_compressed_fastq_cmd;
                } else {
                    # Sequence or quality line.
                    print substr($0, split1_start, split1_length) | write_compressed_fastq_cmd;
                }
            } END {
                 # Close file handle.
                 close(write_compressed_fastq_cmd);
            }'
}

In [6]:
for file in original_fastq/*24010b*R2_001.fastq.gz
do
    echo $file
    newfile=${file%.fastq.gz}.BCONLY.fastq.gz
    split_fastq $file $newfile &
done

original_fastq/HYA__24010b__20210813_384_PBMC_11_S9_R2_001.fastq.gz
[1] 1115


In [7]:
for file in original_fastq/*2beafa*R2_001.fastq.gz
do
    echo $file
    newfile=${file%.fastq.gz}.BCONLY.fastq.gz
    split_fastq $file $newfile &
done

original_fastq/HYA__2beafa__20210813_384_PBMC_12_S10_R2_001.fastq.gz
[2] 1161


In [8]:
for file in original_fastq/*3d6da9*R2_001.fastq.gz
do
    echo $file
    newfile=${file%.fastq.gz}.BCONLY.fastq.gz
    split_fastq $file $newfile &
done

original_fastq/HYA__3d6da9__20210813_384_PBMC_21_S11_R2_001.fastq.gz
[3] 1169


In [9]:
for file in original_fastq/*5028cb*R2_001.fastq.gz
do
    echo $file
    ls $file
    newfile=${file%.fastq.gz}.BCONLY.fastq.gz
    split_fastq $file $newfile &
done

original_fastq/HYA__5028cb__20210813_384_PBMC_22_S12_R2_001.fastq.gz
original_fastq/HYA__5028cb__20210813_384_PBMC_22_S12_R2_001.fastq.gz
[4] 1178


In [21]:
id=SI-NA-B1
ls original_fastq/*$id*UMI2.fastq.gz
for file in original_fastq/*$id*UMI2.fastq.gz
do
    echo $file
    newfile=${file%.fastq.gz}.BCONLY.fastq.gz
    split_fastq $file $newfile &
done

original_fastq/HVYHWDRXY_1_SI-NA-B1_UMI2.fastq.gz
original_fastq/HVYHWDRXY_2_SI-NA-B1_UMI2.fastq.gz
original_fastq/HVYHWDRXY_1_SI-NA-B1_UMI2.fastq.gz
[1] 29664
original_fastq/HVYHWDRXY_2_SI-NA-B1_UMI2.fastq.gz
[2] 29665


In [22]:
id=SI-NA-B4
ls original_fastq/*$id*UMI2.fastq.gz
for file in original_fastq/*$id*UMI2.fastq.gz
do
    echo $file
    newfile=${file%.fastq.gz}.BCONLY.fastq.gz
    split_fastq $file $newfile &
done

original_fastq/HTLJNDRXY_1_SI-NA-B4_UMI2.fastq.gz
original_fastq/HTLJNDRXY_2_SI-NA-B4_UMI2.fastq.gz
original_fastq/HVYHWDRXY_1_SI-NA-B4_UMI2.fastq.gz
original_fastq/HVYHWDRXY_2_SI-NA-B4_UMI2.fastq.gz
original_fastq/HTLJNDRXY_1_SI-NA-B4_UMI2.fastq.gz
[3] 29710
original_fastq/HTLJNDRXY_2_SI-NA-B4_UMI2.fastq.gz
[4] 29711
original_fastq/HVYHWDRXY_1_SI-NA-B4_UMI2.fastq.gz
[5] 29712
original_fastq/HVYHWDRXY_2_SI-NA-B4_UMI2.fastq.gz
[6] 29714


Some of the MDC samples are sequenced on a reverse complement sequencer, therefore the last 16 and not the first 16 bp contain the barcode.

In [1]:
# using start and end coordinates
split_fastq (){
    local input_fastq_filename="${1}";
    local output_fastq_split1_filename="${2}";
    local split1_start=10;
    local split1_end=25;
    zcat "${input_fastq_filename}" \
      | awk \
            -v "split1_start=${split1_start}" \
            -v "split1_end=${split1_end}" \
            -v "output_fastq_split1_filename=${output_fastq_split1_filename}" \
            '
            BEGIN {
                split1_length = split1_end - split1_start + 1;
                write_compressed_fastq_cmd = "pigz -p 2 -6 -c > " output_fastq_split1_filename;
            }
            {
                if (NR % 2 == 1) {
                    # Read name or "+" line.
                    print $0  | write_compressed_fastq_cmd;
                } else {
                    # Sequence or quality line.
                    print substr($0, split1_start, split1_length) | write_compressed_fastq_cmd;
                }
            } END {
                 # Close file handle.
                 close(write_compressed_fastq_cmd);
            }'
}

In [4]:
module load pigz

In [5]:
for file in original_fastq/Barcelona*47*R2_001.fastq.gz
do
    echo $file
    newfile=${file%.fastq.gz}.BCONLY.fastq.gz
    split_fastq $file $newfile &
done

original_fastq/Barcelona_2_C5_047_S2_L004_R2_001.fastq.gz
[1] 19689


In [6]:
for file in original_fastq/Barcelona*46*S1*R2_001.fastq.gz
do
    echo $file
    newfile=${file%.fastq.gz}.BCONLY.fastq.gz
    split_fastq $file $newfile &
done

original_fastq/Barcelona_1_B5_046_S1_L004_R2_001.fastq.gz
[2] 20228


# Merge
After that, we can merge all the fastqs associated with one sample. For example, see all the fastq files associated with `VIB_10xv11_1`

In [30]:
id=1d7912
samplename=VIB_10xv1_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

[33]+  Done                    cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz
original_fastq/BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L001_R1_001.fastq.gz
original_fastq/BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L001_R2_001.fastq.gz
original_fastq/BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L001_R3_001.fastq.gz
original_fastq/BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L002_R1_001.fastq.gz
original_fastq/BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L002_R2_001.fastq.gz
original_fastq/BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S1_L002_R3_001.fastq.gz
original_fastq/BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S2_L001_R1_001.fastq.gz
original_fastq/BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S2_L001_R2_001.fastq.gz
original_fastq/BAP__1d7912__10x_scATAC_PBMC_3030663_3030456_replicate_1_S2_L001_R3_001.fastq.gz
original_fastq/BAP__1d7

In [25]:
id=67114f
samplename=VIB_10xv1_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/BAP__67114f__10x_scATAC_PBMC_3030663_3030456_replicate_2_S23_L001_R1_001.fastq.gz
original_fastq/BAP__67114f__10x_scATAC_PBMC_3030663_3030456_replicate_2_S23_L001_R2_001.fastq.gz
original_fastq/BAP__67114f__10x_scATAC_PBMC_3030663_3030456_replicate_2_S23_L001_R3_001.fastq.gz
original_fastq/BAP__67114f__10x_scATAC_PBMC_3030663_3030456_replicate_2_S23_L002_R1_001.fastq.gz
original_fastq/BAP__67114f__10x_scATAC_PBMC_3030663_3030456_replicate_2_S23_L002_R2_001.fastq.gz
original_fastq/BAP__67114f__10x_scATAC_PBMC_3030663_3030456_replicate_2_S23_L002_R3_001.fastq.gz
original_fastq/BAP__67114f__10x_scATAC_PBMC_3030663_3030456_replicate_2_S24_L001_R1_001.fastq.gz
original_fastq/BAP__67114f__10x_scATAC_PBMC_3030663_3030456_replicate_2_S24_L001_R2_001.fastq.gz
original_fastq/BAP__67114f__10x_scATAC_PBMC_3030663_3030456_replicate_2_S24_L001_R3_001.fastq.gz
original_fastq/BAP__67114f__10x_scATAC_PBMC_3030663_3030456_replicate_2_S24_L002_R1_001.fastq.gz
original_fastq/BAP__67114f__10

In [27]:
id=6d827c
samplename=VIB_10xmultiome_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

[7]   Done                    cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S37_L001_R1_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S37_L001_R2_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S37_L001_R3_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S37_L002_R1_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S37_L002_R2_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S37_L002_R3_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S38_L001_R1_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S38_L001_R2_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S38_L001_R3_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S38_L002_R1_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S38_L002_R2_001.fastq.gz
original_fastq/BAP__6d827c__10x_MO_ATAC_lib_PBMC_2_S

In [30]:
id=750dfc
samplename=VIB_mtscatac_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S25_L001_R1_001.fastq.gz
original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S25_L001_R2_001.fastq.gz
original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S25_L001_R3_001.fastq.gz
original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S25_L002_R1_001.fastq.gz
original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S25_L002_R2_001.fastq.gz
original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S25_L002_R3_001.fastq.gz
original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S26_L001_R1_001.fastq.gz
original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S26_L001_R2_001.fastq.gz
original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S26_L001_R3_001.fastq.gz
original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S26_L002_R1_001.fastq.gz
original_fastq/BAP__750dfc__10x_scATAC_PBMC_Mito_nuclei_prep_1_S26_L002_R2_001.fastq.gz
original_fastq/BAP__750dfc__10x_

In [31]:
id=764061
samplename=STA_10xv11_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/BAP__764061__stanford2_S1_L001_R1_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L001_R2_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L001_R3_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L002_R1_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L002_R2_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L002_R3_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L003_R1_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L003_R2_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L003_R3_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L004_R1_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L004_R2_001.fastq.gz
original_fastq/BAP__764061__stanford2_S1_L004_R3_001.fastq.gz
original_fastq/BAP__764061__stanford2_S2_L001_R1_001.fastq.gz
original_fastq/BAP__764061__stanford2_S2_L001_R2_001.fastq.gz
original_fastq/BAP__764061__stanford2_S2_L001_R3_001.fastq.gz
original_fastq/BAP__764061__stanford2_S2_L002_R1_001.fastq.gz
original

In [32]:
id=7dd154
samplename=STA_10xv11_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/BAP__7dd154__stanford1_S5_L001_R1_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L001_R2_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L001_R3_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L002_R1_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L002_R2_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L002_R3_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L003_R1_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L003_R2_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L003_R3_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L004_R1_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L004_R2_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S5_L004_R3_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S6_L001_R1_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S6_L001_R2_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S6_L001_R3_001.fastq.gz
original_fastq/BAP__7dd154__stanford1_S6_L002_R1_001.fastq.gz
original

In [33]:
id=87a1e2
samplename=VIB_mtscatac_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S29_L001_R1_001.fastq.gz
original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S29_L001_R2_001.fastq.gz
original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S29_L001_R3_001.fastq.gz
original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S29_L002_R1_001.fastq.gz
original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S29_L002_R2_001.fastq.gz
original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S29_L002_R3_001.fastq.gz
original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S30_L001_R1_001.fastq.gz
original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S30_L001_R2_001.fastq.gz
original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S30_L001_R3_001.fastq.gz
original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S30_L002_R1_001.fastq.gz
original_fastq/BAP__87a1e2__10x_scATAC_PBMC_Mito_nuclei_prep_2_S30_L002_R2_001.fastq.gz
original_fastq/BAP__87a1e2__10x_

In [1]:
id=8b510e
samplename=VIB_10xv11_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

ls: cannot access 'original_fastq/*8b510e*': No such file or directory
[1] 334629
cat: 'original_fastq/*8b510e*R1*.fastq.gz': No such file or directory
[2] 334630
[1]   Exit 1                  cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
cat: 'original_fastq/*8b510e*R2*.fastq.gz': No such file or directory
[3] 334631
[2]   Exit 1                  cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz


: 1

In [36]:
id=c0e4b3
samplename=VIB_10xv2_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S5_L001_R1_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S5_L001_R2_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S5_L001_R3_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S6_L001_R1_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S6_L001_R2_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S6_L001_R3_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S7_L001_R1_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S7_L001_R2_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S7_L001_R3_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S8_L001_R1_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S8_L001_R2_001.fastq.gz
original_fastq/BAP__c0e4b3__20220429_10x_scATAC_v2_PBMC_1_S8_L001_R3_001.fastq.gz
[10] 8385
[11] 8

In [37]:
id=c31af6
samplename=VIB_10xmultiome_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S33_L001_R1_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S33_L001_R2_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S33_L001_R3_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S33_L002_R1_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S33_L002_R2_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S33_L002_R3_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S34_L001_R1_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S34_L001_R2_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S34_L001_R3_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S34_L002_R1_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S34_L002_R2_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S34_L002_R3_001.fastq.gz
original_fastq/BAP__c31af6__10x_MO_ATAC_lib_PBMC_1_S35_L001_R1_001.fastq.gz
original_fas

In [38]:
id=ecb164
samplename=VIB_10xv2_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S10_L001_R1_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S10_L001_R2_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S10_L001_R3_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S11_L001_R1_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S11_L001_R2_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S11_L001_R3_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S12_L001_R1_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S12_L001_R2_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S12_L001_R3_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S9_L001_R1_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S9_L001_R2_001.fastq.gz
original_fastq/BAP__ecb164__20220429_10x_scATAC_v2_PBMC_2_S9_L001_R3_001.fastq.gz
[16] 84

In [21]:
id=Barcelona_1
samplename=MDC_mtscatac_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/Barcelona_1_B5_045_S8_L001_R1_001.fastq.gz
original_fastq/Barcelona_1_B5_045_S8_L001_R2_001.BCONLY.fastq.gz
original_fastq/Barcelona_1_B5_045_S8_L001_R2_001.fastq.gz
original_fastq/Barcelona_1_B5_045_S8_L001_R3_001.fastq.gz
original_fastq/Barcelona_1_B5_045_S8_L002_R1_001.fastq.gz
original_fastq/Barcelona_1_B5_045_S8_L002_R2_001.BCONLY.fastq.gz
original_fastq/Barcelona_1_B5_045_S8_L002_R2_001.fastq.gz
original_fastq/Barcelona_1_B5_045_S8_L002_R3_001.fastq.gz
original_fastq/Barcelona_1_B5_046_S1_L004_R1_001.fastq.gz
original_fastq/Barcelona_1_B5_046_S1_L004_R2_001.BCONLY.fastq.gz
original_fastq/Barcelona_1_B5_046_S1_L004_R2_001.fastq.gz
original_fastq/Barcelona_1_B5_046_S1_L004_R3_001.fastq.gz
[1] 17695
[2] 17696
[3] 17697


In [22]:
id=Barcelona_2
samplename=MDC_mtscatac_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/Barcelona_2_C5_046_S9_L001_R1_001.fastq.gz
original_fastq/Barcelona_2_C5_046_S9_L001_R2_001.BCONLY.fastq.gz
original_fastq/Barcelona_2_C5_046_S9_L001_R2_001.fastq.gz
original_fastq/Barcelona_2_C5_046_S9_L001_R3_001.fastq.gz
original_fastq/Barcelona_2_C5_046_S9_L002_R1_001.fastq.gz
original_fastq/Barcelona_2_C5_046_S9_L002_R2_001.BCONLY.fastq.gz
original_fastq/Barcelona_2_C5_046_S9_L002_R2_001.fastq.gz
original_fastq/Barcelona_2_C5_046_S9_L002_R3_001.fastq.gz
original_fastq/Barcelona_2_C5_047_S2_L004_R1_001.fastq.gz
original_fastq/Barcelona_2_C5_047_S2_L004_R2_001.BCONLY.fastq.gz
original_fastq/Barcelona_2_C5_047_S2_L004_R2_001.fastq.gz
original_fastq/Barcelona_2_C5_047_S2_L004_R3_001.fastq.gz
[4] 17704
[5] 17705
[6] 17706


In [1]:
id=Benchmark_1_S1
samplename=BRO_mtscatac_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=Benchmark_2_S2
samplename=BRO_mtscatac_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/Benchmark_1_S1_L001_R1_001.fastq.gz
original_fastq/Benchmark_1_S1_L001_R2_001.fastq.gz
original_fastq/Benchmark_1_S1_L001_R3_001.fastq.gz
original_fastq/Benchmark_1_S1_L002_R1_001.fastq.gz
original_fastq/Benchmark_1_S1_L002_R2_001.fastq.gz
original_fastq/Benchmark_1_S1_L002_R3_001.fastq.gz
original_fastq/Benchmark_1_S1_L003_R1_001.fastq.gz
original_fastq/Benchmark_1_S1_L003_R2_001.fastq.gz
original_fastq/Benchmark_1_S1_L003_R3_001.fastq.gz
original_fastq/Benchmark_1_S1_L004_R1_001.fastq.gz
original_fastq/Benchmark_1_S1_L004_R2_001.fastq.gz
original_fastq/Benchmark_1_S1_L004_R3_001.fastq.gz
[1] 12399
[2] 12400
[3] 12401
original_fastq/Benchmark_2_S2_L001_R1_001.fastq.gz
original_fastq/Benchmark_2_S2_L001_R2_001.fastq.gz
original_fastq/Benchmark_2_S2_L001_R3_001.fastq.gz
original_fastq/Benchmark_2_S2_L002_R1_001.fastq.gz
original_fastq/Benchmark_2_S2_L002_R2_001.fastq.gz
original_fastq/Benchmark_2_S2_L002_R3_001.fastq.gz
original_fastq/Benchmark_2_S2_L003_R1_001.fastq.gz
o

In [2]:
id=BioRad-ATACseq4
samplename=BIO_ddseq_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

id=BioRad-ATACseq5
samplename=BIO_ddseq_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

id=N707-ATACExp55
samplename=BIO_ddseq_3
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

id=N710-ATACExp55
samplename=BIO_ddseq_4
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

original_fastq/BioRad-ATACseq4_S4_L001_R1_001.fastq.gz
original_fastq/BioRad-ATACseq4_S4_L001_R2_001.fastq.gz
original_fastq/BioRad-ATACseq4_S4_L002_R1_001.fastq.gz
original_fastq/BioRad-ATACseq4_S4_L002_R2_001.fastq.gz
[7] 12407
[8] 12408
original_fastq/BioRad-ATACseq5_S5_L001_R1_001.fastq.gz
original_fastq/BioRad-ATACseq5_S5_L001_R2_001.fastq.gz
original_fastq/BioRad-ATACseq5_S5_L002_R1_001.fastq.gz
original_fastq/BioRad-ATACseq5_S5_L002_R2_001.fastq.gz
[9] 12422
[10] 12428
original_fastq/N707-ATACExp55-Sample7_S1_L001_R1_001.fastq.gz
original_fastq/N707-ATACExp55-Sample7_S1_L001_R2_001.fastq.gz
original_fastq/N707-ATACExp55-Sample7_S1_L002_R1_001.fastq.gz
original_fastq/N707-ATACExp55-Sample7_S1_L002_R2_001.fastq.gz
original_fastq/N707-ATACExp55-Sample7_S1_L003_R1_001.fastq.gz
original_fastq/N707-ATACExp55-Sample7_S1_L003_R2_001.fastq.gz
original_fastq/N707-ATACExp55-Sample7_S1_L004_R1_001.fastq.gz
original_fastq/N707-ATACExp55-Sample7_S1_L004_R2_001.fastq.gz
[11] 12430
[12] 12431
o

In [3]:
ls original_fastq/*10000*

original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L001_R1_001.fastq.gz
original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L001_R2_001.fastq.gz
original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L001_R3_001.fastq.gz
original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L002_R1_001.fastq.gz
original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L002_R2_001.fastq.gz
original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L002_R3_001.fastq.gz
original_fastq/EV043_Everest_10000_PBMC_Chromium_X_rep3_1258208_S2_L001_R1_001.fastq.gz
original_fastq/EV043_Everest_10000_PBMC_Chromium_X_rep3_1258208_S2_L001_R2_001.fastq.gz
original_fastq/EV043_Everest_10000_PBMC_Chromium_X_rep3_1258208_S2_L001_R3_001.fastq.gz
original_fastq/EV043_Everest_10000_PBMC_Chromium_X_rep3_1258208_S2_L002_R1_001.fastq.gz
original_fastq/EV043_Everest_10000_PBMC_Chromium_X_rep3_1258208_S2_L002_R2_001.fastq.gz
original_fastq/EV043_Everest_10000_PBMC_Chro

In [4]:
id=EV043_Agora_10000_PBMC_Chromium_X_rep2
samplename=TXG_10xv11_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=EV043_Everest_10000_PBMC_Chromium_X_rep3
samplename=TXG_10xv2_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=EV043_Everest_10000_PBMC_Controller_rep3
samplename=TXG_10xv2_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L001_R1_001.fastq.gz
original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L001_R2_001.fastq.gz
original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L001_R3_001.fastq.gz
original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L002_R1_001.fastq.gz
original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L002_R2_001.fastq.gz
original_fastq/EV043_Agora_10000_PBMC_Chromium_X_rep2_1258199_S1_L002_R3_001.fastq.gz
[15] 12437
[16] 12438
[17] 12439
original_fastq/EV043_Everest_10000_PBMC_Chromium_X_rep3_1258208_S2_L001_R1_001.fastq.gz
original_fastq/EV043_Everest_10000_PBMC_Chromium_X_rep3_1258208_S2_L001_R2_001.fastq.gz
original_fastq/EV043_Everest_10000_PBMC_Chromium_X_rep3_1258208_S2_L001_R3_001.fastq.gz
original_fastq/EV043_Everest_10000_PBMC_Chromium_X_rep3_1258208_S2_L002_R1_001.fastq.gz
original_fastq/EV043_Everest_10000_PBMC_Chromium_X_rep3_1258208_S2_L002_R2_001.fastq.gz
original_fa

# CNAG samples

In [5]:
id=H2TTGBCX3_1
samplename=CNA_ddseq_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

id=H2TTGBCX3_2
samplename=CNA_ddseq_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

original_fastq/H2TTGBCX3_1_N702-NX-xt_S2_L001_R1_001.fastq.gz
original_fastq/H2TTGBCX3_1_N702-NX-xt_S2_L001_R2_001.fastq.gz
[24] 12454
[25] 12455
original_fastq/H2TTGBCX3_2_N702-NX-xt_S2_L002_R1_001.fastq.gz
original_fastq/H2TTGBCX3_2_N702-NX-xt_S2_L002_R2_001.fastq.gz
[26] 12457
[27] 12458


In [1]:
id=HTW7CDRXY_?_SI-NA-A11
samplename=CNA_10xmultiome_1
ls original_fastq/*$id*
cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

id=HTW7CDRXY_?_SI-NA-A12
samplename=CNA_10xmultiome_2
ls original_fastq/*$id*
cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

original_fastq/HTW7CDRXY_1_SI-NA-A11_1.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-A11_2.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-A11_UMI2.BCONLY.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-A11_1.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-A11_2.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-A11_UMI2.BCONLY.fastq.gz
[1] 3665127
[2] 3665128
[3] 3665129
original_fastq/HTW7CDRXY_1_SI-NA-A12_1.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-A12_2.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-A12_UMI2.BCONLY.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-A12_1.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-A12_2.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-A12_UMI2.BCONLY.fastq.gz
[4] 3665131
[5] 3665132
[6] 3665139


!!!!!

In [4]:
id=*SI-NA-B1
samplename=CNA_mtscatac_1
ls original_fastq/*$id*
cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

[3]+  Done                    cat original_fastq/*$id*UMI2.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B1_1.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B1_2.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B1_UMI2.BCONLY.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B1_1.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B1_2.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B1_UMI2.BCONLY.fastq.gz
original_fastq/HVYHWDRXY_1_SI-NA-B1_1.fastq.gz
original_fastq/HVYHWDRXY_1_SI-NA-B1_2.fastq.gz
original_fastq/HVYHWDRXY_1_SI-NA-B1_UMI2.BCONLY.fastq.gz
original_fastq/HVYHWDRXY_2_SI-NA-B1_1.fastq.gz
original_fastq/HVYHWDRXY_2_SI-NA-B1_2.fastq.gz
original_fastq/HVYHWDRXY_2_SI-NA-B1_UMI2.BCONLY.fastq.gz
original_fastq/HW25TDRXY_1_SI-NA-B1_1.fastq.gz
original_fastq/HW25TDRXY_1_SI-NA-B1_2.fastq.gz
original_fastq/HW25TDRXY_1_SI-NA-B1_UMI2.BCONLY.fastq.gz
original_fastq/HW25TDRXY_2_SI-NA-B1_1.fastq.gz
original_fastq/HW25TDRXY_2_SI-NA-B1_2.fastq.gz
original_fastq/HW25TDRXY_2_SI-N

In [9]:
id=HTW7CDRXY_?_SI-NA-B2
samplename=CNA_mtscatac_2
ls original_fastq/*$id*
cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

original_fastq/HTW7CDRXY_1_SI-NA-B2_1.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B2_2.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B2_UMI2.BCONLY.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B2_1.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B2_2.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B2_UMI2.BCONLY.fastq.gz
[37] 12484
[38] 12485
[39] 12486


In [14]:
id=HTW7CDRXY_?_SI-NA-B3
samplename=CNA_10xv2_1
ls original_fastq/*$id*
cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

[28]   Done                    cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
[29]   Done                    cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz
[31]-  Done                    cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
[32]+  Done                    cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B3_1.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B3_2.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B3_UMI2.BCONLY.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B3_1.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B3_2.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B3_UMI2.BCONLY.fastq.gz
[24] 12741
[25] 12742
[26] 12743


In [3]:
id=SI-NA-B4
samplename=CNA_10xv2_2
ls original_fastq/*$id*
cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

original_fastq/HTLJNDRXY_1_SI-NA-B4_1.fastq.gz
original_fastq/HTLJNDRXY_1_SI-NA-B4_2.fastq.gz
original_fastq/HTLJNDRXY_1_SI-NA-B4_UMI2.BCONLY.fastq.gz
original_fastq/HTLJNDRXY_2_SI-NA-B4_1.fastq.gz
original_fastq/HTLJNDRXY_2_SI-NA-B4_2.fastq.gz
original_fastq/HTLJNDRXY_2_SI-NA-B4_UMI2.BCONLY.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B4_1.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B4_2.fastq.gz
original_fastq/HTW7CDRXY_1_SI-NA-B4_UMI2.BCONLY.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B4_1.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B4_2.fastq.gz
original_fastq/HTW7CDRXY_2_SI-NA-B4_UMI2.BCONLY.fastq.gz
original_fastq/HVYHWDRXY_1_SI-NA-B4_1.fastq.gz
original_fastq/HVYHWDRXY_1_SI-NA-B4_2.fastq.gz
original_fastq/HVYHWDRXY_1_SI-NA-B4_UMI2.BCONLY.fastq.gz
original_fastq/HVYHWDRXY_2_SI-NA-B4_1.fastq.gz
original_fastq/HVYHWDRXY_2_SI-NA-B4_2.fastq.gz
original_fastq/HVYHWDRXY_2_SI-NA-B4_UMI2.BCONLY.fastq.gz
original_fastq/HW25TDRXY_1_SI-NA-B4_1.fastq.gz
original_fastq/HW25TDRXY_1_SI-NA-B4_2.fastq.gz


In [16]:
id=*SI-NA-A4*
samplename=CNA_10xv11_3
ls original_fastq/*$id*
cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

original_fastq/HTW7TDRXY_1_SI-NA-A4_1.fastq.gz
original_fastq/HTW7TDRXY_1_SI-NA-A4_2.fastq.gz
original_fastq/HTW7TDRXY_1_SI-NA-A4_UMI2.BCONLY.fastq.gz
original_fastq/HTW7TDRXY_2_SI-NA-A4_1.fastq.gz
original_fastq/HTW7TDRXY_2_SI-NA-A4_2.fastq.gz
original_fastq/HTW7TDRXY_2_SI-NA-A4_UMI2.BCONLY.fastq.gz
original_fastq/HW25TDRXY_1_SI-NA-A4_1.fastq.gz
original_fastq/HW25TDRXY_1_SI-NA-A4_2.fastq.gz
original_fastq/HW25TDRXY_1_SI-NA-A4_UMI2.BCONLY.fastq.gz
original_fastq/HW25TDRXY_2_SI-NA-A4_1.fastq.gz
original_fastq/HW25TDRXY_2_SI-NA-A4_2.fastq.gz
original_fastq/HW25TDRXY_2_SI-NA-A4_UMI2.BCONLY.fastq.gz
[30] 12754
[31] 12755
[32] 12756


In [17]:
id=HTW7TDRXY_?_SI-NA-A5
samplename=CNA_10xv11_4
ls original_fastq/*$id*
cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

id=HTW7TDRXY_?_SI-NA-A6
samplename=CNA_10xv11_5
ls original_fastq/*$id*
cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*2.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2.BCONLY.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

[1]   Done                    cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
original_fastq/HTW7TDRXY_1_SI-NA-A5_1.fastq.gz
original_fastq/HTW7TDRXY_1_SI-NA-A5_2.fastq.gz
original_fastq/HTW7TDRXY_1_SI-NA-A5_UMI2.BCONLY.fastq.gz
original_fastq/HTW7TDRXY_2_SI-NA-A5_1.fastq.gz
original_fastq/HTW7TDRXY_2_SI-NA-A5_2.fastq.gz
original_fastq/HTW7TDRXY_2_SI-NA-A5_UMI2.BCONLY.fastq.gz
[33] 12763
[34] 12764
[35] 12770
original_fastq/HTW7TDRXY_1_SI-NA-A6_1.fastq.gz
original_fastq/HTW7TDRXY_1_SI-NA-A6_2.fastq.gz
original_fastq/HTW7TDRXY_1_SI-NA-A6_UMI2.BCONLY.fastq.gz
original_fastq/HTW7TDRXY_2_SI-NA-A6_1.fastq.gz
original_fastq/HTW7TDRXY_2_SI-NA-A6_2.fastq.gz
original_fastq/HTW7TDRXY_2_SI-NA-A6_UMI2.BCONLY.fastq.gz
[36] 12772
[37] 12773
[38] 12775


In [18]:
id=HK35CDRXX_?_SI-NA-H6
samplename=CNA_10xv11_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

id=HK35CDRXX_?_SI-NA-A7
samplename=CNA_10xv11_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &
cat original_fastq/*$id*UMI2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

original_fastq/HK35CDRXX_1_SI-NA-H6_S2_L001_R1_001.fastq.gz
original_fastq/HK35CDRXX_1_SI-NA-H6_S2_L001_R2_001.fastq.gz
original_fastq/HK35CDRXX_1_SI-NA-H6_S2_L001_UMI2_001.fastq.gz
original_fastq/HK35CDRXX_2_SI-NA-H6_S2_L002_R1_001.fastq.gz
original_fastq/HK35CDRXX_2_SI-NA-H6_S2_L002_R2_001.fastq.gz
original_fastq/HK35CDRXX_2_SI-NA-H6_S2_L002_UMI2_001.fastq.gz
[39] 12777
[40] 12778
[41] 12779
original_fastq/HK35CDRXX_1_SI-NA-A7_S1_L001_R1_001.fastq.gz
original_fastq/HK35CDRXX_1_SI-NA-A7_S1_L001_R2_001.fastq.gz
original_fastq/HK35CDRXX_1_SI-NA-A7_S1_L001_UMI2_001.fastq.gz
original_fastq/HK35CDRXX_2_SI-NA-A7_S3_L002_R1_001.fastq.gz
original_fastq/HK35CDRXX_2_SI-NA-A7_S3_L002_R2_001.fastq.gz
original_fastq/HK35CDRXX_2_SI-NA-A7_S3_L002_UMI2_001.fastq.gz
[42] 12781
[43] 12782
[44] 12783


In [8]:
ls original_fastq_scrna/*H5K*

original_fastq_scrna/CNA_10xmultiomerna_1__H5KYCDMXY_1_SI-TT-A10_1.fastq.gz
original_fastq_scrna/CNA_10xmultiomerna_1__H5KYCDMXY_1_SI-TT-A10_2.fastq.gz
original_fastq_scrna/CNA_10xmultiomerna_1__H5KYCDMXY_1_SI-TT-A11_1.fastq.gz
original_fastq_scrna/CNA_10xmultiomerna_1__H5KYCDMXY_1_SI-TT-A11_2.fastq.gz
original_fastq_scrna/CNA_10xmultiomerna_2__H5KYCDMXY_2_SI-TT-A10_1.fastq.gz
original_fastq_scrna/CNA_10xmultiomerna_2__H5KYCDMXY_2_SI-TT-A10_2.fastq.gz
original_fastq_scrna/CNA_10xmultiomerna_2__H5KYCDMXY_2_SI-TT-A11_1.fastq.gz
original_fastq_scrna/CNA_10xmultiomerna_2__H5KYCDMXY_2_SI-TT-A11_2.fastq.gz


In [4]:
id=H5KYCDMXY_1
samplename=CNA_10xmultiome_1
ls original_fastq_scrna/*$id*
cat original_fastq_scrna/*$id*1.fastq.gz > full_fastq_scrna/${samplename}__R1.FULL.fastq.gz &
cat original_fastq_scrna/*$id*2.fastq.gz > full_fastq_scrna/${samplename}__R2.FULL.fastq.gz &

id=H5KYCDMXY_2
samplename=CNA_10xmultiome_2
ls original_fastq_scrna/*$id*
cat original_fastq_scrna/*$id*1.fastq.gz > full_fastq_scrna/${samplename}__R1.FULL.fastq.gz &
cat original_fastq_scrna/*$id*2.fastq.gz > full_fastq_scrna/${samplename}__R2.FULL.fastq.gz &

[1]   Done                    cat full_fastq_scrna/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
[3]-  Done                    cat full_fastq_scrna/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
full_fastq_scrna/CNA_10xmultiomerna_1__H5KYCDMXY_1_SI-TT-A10_1.fastq.gz
full_fastq_scrna/CNA_10xmultiomerna_1__H5KYCDMXY_1_SI-TT-A10_2.fastq.gz
full_fastq_scrna/CNA_10xmultiomerna_1__H5KYCDMXY_1_SI-TT-A11_1.fastq.gz
full_fastq_scrna/CNA_10xmultiomerna_1__H5KYCDMXY_1_SI-TT-A11_2.fastq.gz
[5] 1237495
[6] 1237496
full_fastq_scrna/CNA_10xmultiomerna_2__H5KYCDMXY_2_SI-TT-A10_1.fastq.gz
full_fastq_scrna/CNA_10xmultiomerna_2__H5KYCDMXY_2_SI-TT-A10_2.fastq.gz
full_fastq_scrna/CNA_10xmultiomerna_2__H5KYCDMXY_2_SI-TT-A11_1.fastq.gz
full_fastq_scrna/CNA_10xmultiomerna_2__H5KYCDMXY_2_SI-TT-A11_2.fastq.gz
[7] 1237498
[8] 1237499


In [10]:
ls original_fastq_scrna/VIB_10xmultiome*

original_fastq_scrna/VIB_10xmultiomerna_1__L001_R1.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_1__L001_R3.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_1__L002_R1.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_1__L002_R3.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_2__L001_R1.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_2__L001_R3.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_2__L002_R1.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_2__L002_R3.FULL.fastq.gz


In [13]:
id=VIB_10xmultiomerna_1
samplename=VIB_10xmultiomerna_1
ls original_fastq_scrna/*$id*
cat original_fastq_scrna/*$id*1.FULL.fastq.gz > full_fastq_scrna/${samplename}__R1.FULL.fastq.gz &
cat original_fastq_scrna/*$id*3.FULL.fastq.gz > full_fastq_scrna/${samplename}__R2.FULL.fastq.gz &

id=VIB_10xmultiomerna_2
samplename=VIB_10xmultiomerna_2
ls original_fastq_scrna/*$id*
cat original_fastq_scrna/*$id*1.FULL.fastq.gz > full_fastq_scrna/${samplename}__R1.FULL.fastq.gz &
cat original_fastq_scrna/*$id*3.FULL.fastq.gz > full_fastq_scrna/${samplename}__R2.FULL.fastq.gz &

[1]+  Done                    cat original_fastq_scrna/*$id*1.FULL.fastq.gz > full_fastq_scrna/${samplename}__R1.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_1__L001_R1.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_1__L001_R3.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_1__L002_R1.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_1__L002_R3.FULL.fastq.gz
[1] 1239246
[2] 1239247
original_fastq_scrna/VIB_10xmultiomerna_2__L001_R1.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_2__L001_R3.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_2__L002_R1.FULL.fastq.gz
original_fastq_scrna/VIB_10xmultiomerna_2__L002_R3.FULL.fastq.gz
[3] 1239249
[4] 1239250


In [53]:
id=24010b
samplename=VIB_hydrop_11
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=2beafa
samplename=VIB_hydrop_12
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=3d6da9
samplename=VIB_hydrop_21
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=5028cb
samplename=VIB_hydrop_22
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

[1]   Done                    split_fastq $file $newfile
[2]   Done                    split_fastq $file $newfile
[4]   Done                    split_fastq $file $newfile
[5]-  Done                    split_fastq $file $newfile
[6]+  Done                    split_fastq $file $newfile
original_fastq/HYA__24010b__20210813_384_PBMC_11_S9_R1_001.fastq.gz
original_fastq/HYA__24010b__20210813_384_PBMC_11_S9_R2_001.fastq.gz
original_fastq/HYA__24010b__20210813_384_PBMC_11_S9_R3_001.fastq.gz
[1] 15585
[2] 15602
[3] 15607
original_fastq/HYA__2beafa__20210813_384_PBMC_12_S10_R1_001.fastq.gz
original_fastq/HYA__2beafa__20210813_384_PBMC_12_S10_R2_001.fastq.gz
original_fastq/HYA__2beafa__20210813_384_PBMC_12_S10_R3_001.fastq.gz
[4] 15714
[5] 15725
[6] 15748
original_fastq/HYA__3d6da9__20210813_384_PBMC_21_S11_R1_001.fastq.gz
original_fastq/HYA__3d6da9__20210813_384_PBMC_21_S11_R2_001.fastq.gz
original_fastq/HYA__3d6da9__20210813_384_PBMC_21_S11_R3_001.fastq.gz
[7] 15891
[8] 15931
[9] 15956
origina

In [21]:
id=*PBMC_1_S1_
samplename=HAR_ddseq_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

id=*PBMC_2_S2_
samplename=HAR_ddseq_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

[3]   Done                    cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz
[4]   Done                    cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
[6]   Done                    cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz
[10]   Done                    cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz
[11]   Done                    cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
[13]   Done                    cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
[16]   Done                    cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz
[22]   Done                    cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz
[24]   Done                    cat original_fastq/*$id*1.fastq.gz > full_fastq/${samplename}__R1.FU

In [23]:
id=Sanger_1_
samplename=SAN_10xmultiome_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=Sanger_2_
samplename=SAN_10xmultiome_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

[18]   Done                    cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
[20]   Done                    cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz
original_fastq/Sanger_1_S1_L001_R1_001.fastq.gz
original_fastq/Sanger_1_S1_L001_R2_001.fastq.gz
original_fastq/Sanger_1_S1_L001_R3_001.fastq.gz
original_fastq/Sanger_1_S1_L002_R1_001.fastq.gz
original_fastq/Sanger_1_S1_L002_R2_001.fastq.gz
original_fastq/Sanger_1_S1_L002_R3_001.fastq.gz
original_fastq/Sanger_1_S1_L003_R1_001.fastq.gz
original_fastq/Sanger_1_S1_L003_R2_001.fastq.gz
original_fastq/Sanger_1_S1_L003_R3_001.fastq.gz
original_fastq/Sanger_1_S1_L004_R1_001.fastq.gz
original_fastq/Sanger_1_S1_L004_R2_001.fastq.gz
original_fastq/Sanger_1_S1_L004_R3_001.fastq.gz
[28] 13453
[29] 13454
[30] 13455
original_fastq/Sanger_2_S1_L005_R1_001.fastq.gz
original_fastq/Sanger_2_S1_L005_R2_001.fastq.gz
original_fastq/Sanger_2_S1_L005_R3_001.fastq.gz
original_fastq/Sanger

In [25]:
id=a659d1
samplename=EPF_hydrop_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=aa4c8e
samplename=EPF_hydrop_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=b4c775
samplename=EPF_hydrop_3
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=bcae2e
samplename=EPF_hydrop_4
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R3*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/BAP__64c29e__EPF_hydrop_21_RESEQUENCED_S1_R1_001.fastq.gz
original_fastq/BAP__64c29e__EPF_hydrop_21_RESEQUENCED_S1_R2_001.fastq.gz
original_fastq/BAP__64c29e__EPF_hydrop_21_RESEQUENCED_S1_R3_001.fastq.gz
original_fastq/BAP__64c29e__EPF_hydrop_21_S5_R1_001.fastq.gz
original_fastq/BAP__64c29e__EPF_hydrop_21_S5_R2_001.fastq.gz
original_fastq/BAP__64c29e__EPF_hydrop_21_S5_R3_001.fastq.gz
[34] 13476
[35] 13477
[36] 13478
original_fastq/BAP__6f3312__EPF_hydrop_22_RESEQUENCED_S2_R1_001.fastq.gz
original_fastq/BAP__6f3312__EPF_hydrop_22_RESEQUENCED_S2_R2_001.fastq.gz
original_fastq/BAP__6f3312__EPF_hydrop_22_RESEQUENCED_S2_R3_001.fastq.gz
original_fastq/BAP__6f3312__EPF_hydrop_22_S6_R1_001.fastq.gz
original_fastq/BAP__6f3312__EPF_hydrop_22_S6_R2_001.fastq.gz
original_fastq/BAP__6f3312__EPF_hydrop_22_S6_R3_001.fastq.gz
[37] 13480
[38] 13481
[39] 13482
original_fastq/BAP__96c271__EPF_hydrop_23_RESEQUENCED_S3_R1_001.fastq.gz
original_fastq/BAP__96c271__EPF_hydrop_23_RESEQUENCED_S3_

In [26]:
id=HTMHHDRXY*52*
samplename=CNA_hydrop_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*UMI2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=HTMHHDRXY*53*
samplename=CNA_hydrop_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*UMI2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*/R2*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

id=HTMHHDRXY*55*
samplename=CNA_hydrop_3
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*UMI2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &
cat original_fastq/*$id*/R2*.fastq.gz > full_fastq/${samplename}__R3.FULL.fastq.gz &

original_fastq/HTMHHDRXY_1_HYi7-ATAC-384-i7-52_S5_L001_R1_001.fastq.gz
original_fastq/HTMHHDRXY_1_HYi7-ATAC-384-i7-52_S5_L001_R2_001.fastq.gz
original_fastq/HTMHHDRXY_1_HYi7-ATAC-384-i7-52_S5_L001_UMI2_001.fastq.gz
original_fastq/HTMHHDRXY_2_HYi7-ATAC-384-i7-52_S2_L002_R1_001.fastq.gz
original_fastq/HTMHHDRXY_2_HYi7-ATAC-384-i7-52_S2_L002_R2_001.fastq.gz
original_fastq/HTMHHDRXY_2_HYi7-ATAC-384-i7-52_S2_L002_UMI2_001.fastq.gz
[58] 13518
[59] 13524
[60] 13525
original_fastq/HTMHHDRXY_1_HYi7-ATAC-384-i7-53_S6_L001_R1_001.fastq.gz
original_fastq/HTMHHDRXY_1_HYi7-ATAC-384-i7-53_S6_L001_R2_001.fastq.gz
original_fastq/HTMHHDRXY_1_HYi7-ATAC-384-i7-53_S6_L001_UMI2_001.fastq.gz
original_fastq/HTMHHDRXY_2_HYi7-ATAC-384-i7-53_S1_L002_R1_001.fastq.gz
original_fastq/HTMHHDRXY_2_HYi7-ATAC-384-i7-53_S1_L002_R2_001.fastq.gz
original_fastq/HTMHHDRXY_2_HYi7-ATAC-384-i7-53_S1_L002_UMI2_001.fastq.gz
[61] 13527
[62] 13528
[63] 13529
original_fastq/HTMHHDRXY_1_HYi7-ATAC-384-i7-54_S1_L001_R1_001.fastq.gz
ori

In [6]:
id=Pillai-ZD-3904-03-C
samplename=UCS_ddseq_1
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

id=Pillai-ZD-3904-04-D
samplename=UCS_ddseq_2
ls original_fastq/*$id*
cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz &
cat original_fastq/*$id*R2*.fastq.gz > full_fastq/${samplename}__R2.FULL.fastq.gz &

original_fastq/Pillai-ZD-3904-03-C_S1_L001_R1_001.fastq.gz
original_fastq/Pillai-ZD-3904-03-C_S1_L001_R2_001.fastq.gz
original_fastq/Pillai-ZD-3904-03-C_S1_L002_R1_001.fastq.gz
original_fastq/Pillai-ZD-3904-03-C_S1_L002_R2_001.fastq.gz
original_fastq/Pillai-ZD-3904-03-C_S1_L003_R1_001.fastq.gz
original_fastq/Pillai-ZD-3904-03-C_S1_L003_R2_001.fastq.gz
original_fastq/Pillai-ZD-3904-03-C_S1_L004_R1_001.fastq.gz
original_fastq/Pillai-ZD-3904-03-C_S1_L004_R2_001.fastq.gz
[1] 12221
[2] 12222
ls: cannot access original_fastq/*Pillai-ZD-3904-03-D*: No such file or directory
[3] 12224
cat: original_fastq/*Pillai-ZD-3904-03-D*R1*.fastq.gz: No such file or directory
[3]+  Exit 1                  cat original_fastq/*$id*R1*.fastq.gz > full_fastq/${samplename}__R1.FULL.fastq.gz
[3] 12225


: 1